## 0. Import Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import preprocessing

In [2]:
df = pd.read_csv('data/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv')
df2 = pd.read_csv('data/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv')

## 1.  Train Test Split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df, df2, test_size = 0.25, random_state = 47)

## 2. Missing Values

__Missing Values Decisions__
- drop scheme_name column
- drop funder column, based on research it is unreliable and highly varied
- drop NAN rows from scheme_management, funder, installer, subvillage, permit 

__Non-Relevant Data__
- num_private has no description and is all zeros, removing column. 
- drop payment_type, quantity group, waterpoint_type_group because these are redundant with payment, quantity.

__Categorical Features__
- All except latitude, longitude, gps height, population, construction_year

__Rows__
- we decided not to drop any rows 

In [4]:
#dropping columns
X_train = X_train.drop(columns=['scheme_name', 'payment_type', 'quantity_group', 'waterpoint_type_group', 'num_private'])

### Imputing

In [5]:
X_train.isna().sum()

id                          0
amount_tsh                  0
date_recorded               0
funder                   2720
gps_height                  0
installer                2740
longitude                   0
latitude                    0
wpt_name                    0
basin                       0
subvillage                286
region                      0
region_code                 0
district_code               0
lga                         0
ward                        0
population                  0
public_meeting           2508
recorded_by                 0
scheme_management        2938
permit                   2263
construction_year           0
extraction_type             0
extraction_type_group       0
extraction_type_class       0
management                  0
management_group            0
payment                     0
water_quality               0
quality_group               0
quantity                    0
source                      0
source_type                 0
source_cla

In [6]:
cols = list(X_train.columns)

In [8]:
SI = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_train_imputed = SI.fit_transform(X_train)
X_train_imputed

array([[29040, 0.0, '2013-02-05', ..., 'spring', 'groundwater',
        'improved spring'],
       [69788, 50.0, '2011-03-12', ..., 'river/lake', 'surface',
        'communal standpipe'],
       [51208, 300.0, '2011-02-26', ..., 'borehole', 'groundwater',
        'communal standpipe'],
       ...,
       [53875, 0.0, '2013-03-15', ..., 'borehole', 'groundwater',
        'communal standpipe'],
       [8608, 20000.0, '2011-02-28', ..., 'spring', 'groundwater',
        'communal standpipe'],
       [60477, 0.0, '2011-03-14', ..., 'borehole', 'groundwater',
        'communal standpipe']], dtype=object)

In [9]:
X_train_imputed = pd.DataFrame(data=X_train_imputed, columns=cols)
X_train_imputed.head(3)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,...,management,management_group,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type
0,29040,0,2013-02-05,Tcrs,1295,Community,30.6172,-3.57407,Ndagije,Lake Tanganyika,...,vwc,user-group,unknown,soft,good,enough,spring,spring,groundwater,improved spring
1,69788,50,2011-03-12,Private Individual,181,WU,38.354,-6.64235,Digali,Wami / Ruvu,...,private operator,commercial,pay per bucket,soft,good,enough,river,river/lake,surface,communal standpipe
2,51208,300,2011-02-26,Ki,490,Ki,37.0489,-6.75865,Shuleni,Wami / Ruvu,...,vwc,user-group,pay when scheme fails,soft,good,insufficient,machine dbh,borehole,groundwater,communal standpipe


In [10]:
X_train_imputed.isna().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
water_quality            0
quality_group            0
quantity                 0
source                   0
source_type              0
source_class             0
waterpoint_type          0
dtype: int64

### Change Datatypes

In [11]:
#change date to datetime format
X_train.date_recorded = pd.to_datetime(X_train.date_recorded)

In [12]:
X_train.dtypes

id                                int64
amount_tsh                      float64
date_recorded            datetime64[ns]
funder                           object
gps_height                        int64
installer                        object
longitude                       float64
latitude                        float64
wpt_name                         object
basin                            object
subvillage                       object
region                           object
region_code                       int64
district_code                     int64
lga                              object
ward                             object
population                        int64
public_meeting                   object
recorded_by                      object
scheme_management                object
permit                           object
construction_year                 int64
extraction_type                  object
extraction_type_group            object
extraction_type_class            object


In [81]:
# creating a list of non-numeric columns
#dropped wpt_name

obj_cols = ['basin', 'region', 'lga', 
              'public_meeting', 'recorded_by', 'scheme_management', 'permit', 
               'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 
               'management_group', 'payment', 'water_quality', 'quality_group', 
               'quantity', 'source', 'source_type', 'source_class', 
               'waterpoint_type']

In [71]:
length_list = []

for item in obj_cols:
    
    length_list.append(item + ":" + str(len(X_train[item].unique())))
    
length_list   

['basin:9',
 'region:21',
 'lga:124',
 'public_meeting:3',
 'recorded_by:1',
 'scheme_management:13',
 'permit:3',
 'extraction_type:18',
 'extraction_type_group:13',
 'extraction_type_class:7',
 'management:12',
 'management_group:5',
 'payment:7',
 'water_quality:8',
 'quality_group:6',
 'quantity:5',
 'source:10',
 'source_type:7',
 'source_class:3',
 'waterpoint_type:7']

In [14]:
#creating a list of numeric columns
num_cols = ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 
            'population', 'construction_year', 'id_code']

In [15]:
#changing non-numeric columns to strings
for col in obj_cols:
            X_train[col] = X_train[col].astype(str)

In [16]:
#doing the same code as the cell above in a function
def change_column_type(dataframe, col_list):
    """Change certain columns in a dataframe to string
    col_list = the columns to be changed
    """
    for col in col_list:
            dataframe[col] = dataframe[col].astype(str)

In [17]:
# return the name of the feature if the value of the name in the dtype list is object
[value for value in X_train.dtypes if type(value) == object]
# the empty list is saying that there are no object types, they have been changed to strings.

[]

## One-Hot Encoding

### One-Hot X_train

In [72]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown='error')
encoded_cols = ohe.fit_transform(X_train[obj_cols])

#make dataframe out of one-hot encoded colums
df_ohe = pd.DataFrame(encoded_cols)
df_ohe.head(3)

,0,1,2,3,4,5,6,7,8,9,...,252,253,254,255,256,257,258,259,260,261
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [73]:
#make dataframe out of numeric columns
df_num_cols = X_train[['amount_tsh', 'gps_height', 'longitude', 'latitude',
                       'region_code', 'district_code', 'population', 'construction_year']]
df_num_cols = df_num_cols.reset_index()
df_num_cols.head(3)

,index,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year
0,8956,0.0,1295,30.617206,-3.574069,16,1,120,2001
1,16599,50.0,181,38.354049,-6.642347,6,1,50,2011
2,53788,300.0,490,37.048901,-6.758648,5,1,500,2011


In [76]:
#merge numeric columns and one-hot encoded columns

X_train_ohe = df_ohe.merge(df_num_cols, how='outer', left_index=True, right_index=True)
X_train_ohe.head(3)

,0,1,2,3,4,5,6,7,8,9,...,261,index,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8956,0.0,1295,30.617206,-3.574069,16,1,120,2001
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,16599,50.0,181,38.354049,-6.642347,6,1,50,2011
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,53788,300.0,490,37.048901,-6.758648,5,1,500,2011


### One-Hot X_test

#### Function for Imputing

In [77]:
def impute_dataframe(dataframe):
    """Function takes in a dataframe and uses simple imputer to insert the 
    most frequent values.
    column_list = list of columns in the dataframe
    """
    
    cols = list(dataframe.columns)
   
    from sklearn.impute import SimpleImputer  
    
    SI = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    
    df_imputed = SI.fit_transform(dataframe)

    df_imputed = pd.DataFrame(data=df_imputed, columns=cols)

    return df_imputed

In [79]:
X_test_imputed = impute_dataframe(X_test) 

In [84]:
#One hot encode X_test without a function

X_test_ohe1 = ohe.fit_transform(X_test_imputed[obj_cols])

#make dataframe out of one-hot encoded columns
X_test_ohe = pd.DataFrame(X_test_ohe1)


#make dataframe out of numeric columns
X_test_num_cols = X_test[['amount_tsh', 'gps_height', 'longitude', 'latitude',
                       'region_code', 'district_code', 'population', 'construction_year']]
X_test_num_cols = X_test_num_cols.reset_index()

#merge numeric columns and one-hot encoded columns
X_test_ohe = X_test_ohe.merge(X_test_num_cols, how='outer', left_index=True, right_index=True)

X_test_ohe.head(3)

,0,1,2,3,4,5,6,7,8,9,...,257,index,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,56122,0.0,0,35.623797,-6.047888,1,5,0,0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,37038,0.0,0,33.161380,-9.123583,12,6,0,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,52894,0.0,0,33.902498,-9.626092,12,3,0,0


### One-Hot Function

In [43]:
#function to one-hot encode a dataframe

def one_hot(dataframe, n_cols, o_cols):
    """Function to one-hot encode a dataframe given the a list of numeric columns 
    and a list of non-numeric columns.
    n_cols = numeric columns, these do not get one-hot encoded
    o_cols = non-numeric columns, these get one-hot encoded
    """
    
    from sklearn.preprocessing import OneHotEncoder
    
    ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown='error')
    encoded_cols = ohe.fit_transform(dataframe[o_cols])

    df_ohe = pd.DataFrame(encoded_cols)
    
    df_num_cols = dataframe[[n_cols]].reset_index()
       
    df_merged = df_ohe.merge(df_num_cols, how='outer', left_index=True, right_index=True)
    return df_merged

In [54]:
num_cols2 = ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 
            'population', 'construction_year', 'id_code']

obj_cols2 = ['installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 
              'public_meeting', 'recorded_by', 'scheme_management', 'permit', 
               'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 
               'management_group', 'payment', 'water_quality', 'quality_group', 
               'quantity', 'source', 'source_type', 'source_class', 
               'waterpoint_type']

X_test_ohe2 = one_hot(X_test_imputed, num_cols2, obj_cols2)
X_test_ohe2.head(3)

KeyError: "None of [Index([('amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year', 'id_code')], dtype='object')] are in the [columns]"

## Export Pickle File

In [85]:
import pickle
X_train_ohe.to_pickle('X_train_ohe.pickle')
X_test_ohe.to_pickle('X_test_ohe.pickle')